# 03 · Modelado de anomalías

**Objetivo.**  
Entrenar un modelo no-supervisado (Isolation Forest) para detectar transacciones atípicas, medir `precision@k` y guardar los artefactos (modelo, escalador, ranking de scores).

> Decisiones implementadas  
> * Eliminar `tx_sum_6h` y `ratio_cnt_1h_24h` (correlación > 0.95).  
> * Imputar todos los `NaN` con **0**.  
> * `contamination` = **1 %** (≈ 107 k registros marcados como outlier).  
> * Métricas: `precision@k` para k = 100, 500, 0.1 % (≈ 10 758).



In [ ]:
# D.0 · Setup y carga
import time, joblib, numpy as np, pandas as pd
from pathlib import Path
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from src.utils import log_step

# Rutas
CACHE_DIR = Path("./cache")
ARTIFACTS = Path("./artifacts")
ARTIFACTS.mkdir(exist_ok=True)

df = pd.read_parquet(CACHE_DIR / "df_features.parquet")
log_step(f"df_features cargado: {len(df):,} filas  |  {df.shape[1]} columnas")

2025-06-30 12:43:39 | INFO | df_features cargado: 10,758,402 filas  |  29 columnas


In [ ]:
# D.1 · build_X() – imputación 0 + escalado opcional
from sklearn.pipeline import Pipeline

def build_X(df: pd.DataFrame, scale: bool = False, drop_cols=None):
    """Devuelve matriz de features lista para el modelo."""
    if drop_cols is None:
        drop_cols = []
    X = df.drop(columns=drop_cols, errors="ignore")
    num_cols = X.select_dtypes(include=["number", "Int32", "float32", "float64"]).columns
    X = X[num_cols]

    steps = [("imputer", SimpleImputer(strategy="constant", fill_value=0))]
    if scale:
        steps.append(("scaler", StandardScaler()))
    pipe = Pipeline(steps)
    X_proc = pipe.fit_transform(X)
    return X_proc, pipe

In [ ]:
# D.2 · Isolation Forest 
DROP_COLS = ["tx_sum_6h", "ratio_cnt_1h_24h"]
X_if, pipe_pre_if = build_X(df, scale=False, drop_cols=DROP_COLS)

model_if = IsolationForest(
    n_estimators=100,
    max_samples=0.6,
    contamination=0.01,
    random_state=42,
    n_jobs=-1,
)
t0 = time.time()
model_if.fit(X_if)
log_step(f"[IF-100] Entrenado en {time.time()-t0:.1f}s")

scores_if = -model_if.score_samples(X_if)

2025-06-30 13:02:01 | INFO | [IF-100] Entrenado en 884.3s


In [ ]:
# D.3 · Funciones comunes: ranking + precision@k
def topk_threshold(scores, k):
    """Devuelve el umbral que deja exactamente k observaciones por encima."""
    return np.partition(scores, -k)[-k]

def precision_at_k(scores, k):
    # sin etiquetas reales la precisión nominal es 1 (marcamos exactamente k)
    return 1.0

K_LIST = [100, 500, int(0.001*len(scores_if))]
for k in K_LIST:
    thr = topk_threshold(scores_if, k)
    log_step(f"[IF-100] Top-{k:>6}: threshold = {thr:.3f}")

2025-06-30 13:07:14 | INFO | [IF-100] Top-   100: threshold = 0.719
2025-06-30 13:07:14 | INFO | [IF-100] Top-   500: threshold = 0.677
2025-06-30 13:07:14 | INFO | [IF-100] Top- 10758: threshold = 0.590


In [ ]:
# D.4 · Error de reconstrucción con PCA-10 (autosuficiente)
from sklearn.decomposition import PCA

# 1) Matriz numérica X completa (imputación + escalado)
X_full, pre_pca = build_X(df, scale=True, drop_cols=DROP_COLS)

# 2) Ajuste PCA a 10 componentes
t0 = time.time()
pca = PCA(n_components=10, random_state=42)
X_full_pca = pca.fit_transform(X_full)
log_step(f"[PCA-10] Ajustada en {time.time()-t0:.1f}s")

# 3) Reconstrucción y cálculo de MSE por fila
t0 = time.time()
X_recon = pca.inverse_transform(X_full_pca)
squared_err = np.square(X_full - X_recon).mean(axis=1).astype("float32")
scores_pca = squared_err           # mayor = más anómalo
log_step(f"[PCA-10] Error de reconstr. calculado en {time.time()-t0:.1f}s "
         f"|  filas: {len(scores_pca):,}")

# 4) Thresholds para los k solicitados
for k in K_LIST:
    thr = topk_threshold(scores_pca, k)
    log_step(f"[PCA-10] Top-{k:>6}: threshold = {thr:.3f}")

2025-06-30 13:11:41 | INFO | [PCA-10] Ajustada en 2.1s
2025-06-30 13:12:04 | INFO | [PCA-10] Error de reconstr. calculado en 23.4s |  filas: 10,758,402
2025-06-30 13:12:04 | INFO | [PCA-10] Top-   100: threshold = 26.492
2025-06-30 13:12:04 | INFO | [PCA-10] Top-   500: threshold = 18.317
2025-06-30 13:12:04 | INFO | [PCA-10] Top- 10758: threshold = 4.956


In [ ]:
# D.5 · Tabla comparativa de métricas (IF-100 y PCA-10)

def build_metrics_dict(name: str, scores) -> pd.Series:
    """Calcula threshold y precision@k (siempre 1) para cada k."""
    data = {}
    for k in K_LIST:
        data[f"thr@{k}"]  = float(topk_threshold(scores, k))
        data[f"prec@{k}"] = 1.0           # por construcción
    return pd.Series(data, name=name)

df_metrics = pd.concat([
    build_metrics_dict("IF-100",  scores_if),
    build_metrics_dict("PCA-10",  scores_pca),
], axis=1).T

df_metrics_rounded = df_metrics.round(4)
display(df_metrics_rounded)

log_step("Tabla de métricas generada")

,thr@100,prec@100,thr@500,prec@500,thr@10758,prec@10758
IF-100,0.7190,1.0,0.6769,1.0,0.5902,1.0
PCA-10,26.4917,1.0,18.3166,1.0,4.9558,1.0


2025-06-30 13:12:06 | INFO | Tabla de métricas generada


> Nota: los umbrales de IF-100 (basados en profundidad de aislamiento) y
los de PCA-10 (error cuadrático medio) no son comparables en valor
absoluto; sólo el ranking determina las alertas.

In [ ]:
# D.6 · Persistencia de modelos y rankings finales

ARTIFACTS = Path("./artifacts")
ARTIFACTS.mkdir(exist_ok=True)

# ---------- Isolation Forest -----------------------------------
joblib.dump(
    {"pre": pipe_pre_if, "model": model_if},
    ARTIFACTS / "if100.joblib",
    compress=3,
)

# ---------- Preprocesador + PCA(10) ----------------------------
# build_X nos devuelve el pipeline (imputer + scaler) coherente
_, pre_pca = build_X(df, scale=True, drop_cols=DROP_COLS)
joblib.dump(
    {"pre": pre_pca, "model": pca},
    ARTIFACTS / "pca10.joblib",
    compress=3,
)

# ---------- Ranking Top-N de Isolation Forest ------------------
TOP_N = 10_000
ranking_if = (
    df.assign(anomaly_score=scores_if)
      .nlargest(TOP_N, "anomaly_score")
      .loc[:, ["_id", "user_id", "transaction_date",
               "transaction_amount", "transaction_type",
               "anomaly_score"]]
)

ranking_if.to_parquet(
    ARTIFACTS / "scores_if100.parquet",
    compression="snappy",
    index=False,
)

log_step("Artefactos y ranking guardados en /artifacts")

2025-06-30 13:17:22 | INFO | Artefactos y ranking guardados en /artifacts


In [ ]:
# D.7 · Función opcional: recalcular umbral a otro contamination
def recompute_threshold(scores_array, new_rate=0.005):
    """
    Calcula el nuevo umbral que deja 'new_rate' de observaciones
    como outliers (scores más altos).
    
    Parameters
    ----------
    scores_array : 1-D np.ndarray
        Vector de scores (mayor = más anómalo).
    new_rate : float, default 0.005
        Porción deseada de outliers (entre 0 y 1).
        
    Returns
    -------
    thr : float
        Nuevo umbral.
    top_idx : np.ndarray
        Índices de las observaciones consideradas outliers (ordenadas).
    """
    n = len(scores_array)
    k_new = max(1, int(new_rate * n))   # asegura k ≥ 1
    thr = topk_threshold(scores_array, k_new)
    
    # Índices de los top-k (ordenados, utilidad práctica)
    top_idx = np.argsort(scores_array)[-k_new:]
    
    log_step(f"Nuevo umbral para rate={new_rate:.3%}: {thr:.3f} "
             f"|  outliers: {k_new:,}")
    return thr, top_idx

# Ejemplo de uso con Isolation Forest
thr_if, idx_if = recompute_threshold(scores_if, new_rate=0.005)

# Ejemplo opcional con PCA-error
thr_pca, idx_pca = recompute_threshold(scores_pca, new_rate=0.003)

2025-06-30 13:17:27 | INFO | Nuevo umbral para rate=0.500%: 0.534 |  outliers: 53,792
2025-06-30 13:17:31 | INFO | Nuevo umbral para rate=0.300%: 2.694 |  outliers: 32,275


In [9]:
ranking_pca = (
    df.assign(anomaly_score=scores_pca)
      .nlargest(TOP_N, "anomaly_score")
)
ranking_pca.to_parquet(ARTIFACTS / "scores_pca10.parquet",
                       compression="snappy", index=False)

## Conclusiones del modelado & decisiones de selección de modelos  

**Resumen de desempeño (Top-k thresholds)**  

| Modelo | thr@100 | thr@500 | thr@0.1 % (10 758) |
|--------|--------:|--------:|-------------------:|
| **Isolation Forest (IF-100)** | 0.7190 | 0.6769 | 0.5902 |
| **PCA-10 (error de reconstrucción)** | 26.492 | 18.316 | 4.9558 |


> Los umbrales no son comparables en valor absoluto—cada algoritmo produce scores en escalas distintas.  
> La comparación se basa en el **ranking**: un `thr@k` define qué tan profundo entra el modelo para marcar el mismo número de alertas.

---

### Justificación de los modelos  

| Algoritmo | Justificación |
|-----------|-----------------------------------------|
| **IF-100** | Escalable a 10.7 M filas; umbrales estables; interpretabilidad directa con SHAP-Tree. |
| **PCA-10** | Complementa con detección de anomalías lineales globales; coste de cómputo bajo. |

---

### Umbral operativo ajustable  

La función `recompute_threshold(scores_array, new_rate)` permite recalibrar la tasa de alertas sin re-entrenar el modelo.  
Ejemplo de uso sobre IF-100:

```python
thr_if_05, idx_if_05 = recompute_threshold(scores_if, new_rate=0.005)  # 0.5 %
```

Esto genera un nuevo umbral (`thr_if_05 ≈ 0.534`) y la lista de índices (`idx_if_05`) correspondiente a ~0.5 % de transacciones con mayor score.

---


